# 20240921

"MODULE/09.5 DE-MRC 모듈만.ipynb"  파일을 그대로 붙여넣음.



In [ ]:

def kmrc_decorrect(df_rawdata, df_mrc_input):
    # UNIQUE_ID별로 그룹화
    grouped = df_rawdata.groupby('UNIQUE_ID')

    # 예측 결과를 저장할 리스트
    mrc_list = []

    for unique_id, group in grouped:
        stepseq = group['STEPSEQ']
        lot_id = group['LOT_ID']
        wafer = group['Wafer']
        p_eqpid = group['P_EQPID']
        photo_ppid = group['Photo_PPID']
        p_time = group['P_TIME']
        m_time = group['M_TIME']
        chuckid = group['ChuckID']
        reticleid = group['ReticleID']
        base_eqp1 = group['Base_EQP1']
        test = group['TEST']
        die_x = group['DieX']
        die_y = group['DieY']
        mrc_x = group['MRC_X']
        mrc_y = group['MRC_Y']
        step_pitch_x = group['STEP_PITCH_X']
        step_pitch_y = group['STEP_PITCH_Y']
        map_shift_x = group['MAP_SHIFT_X']
        map_shift_y = group['MAP_SHIFT_Y']
        coordinate_x = group['coordinate_X']
        coordinate_y = group['coordinate_Y']   
        wf_x = group['wf_x']
        wf_y = group['wf_y']        
        x_reg = group['X_reg']
        y_reg = group['Y_reg']
        pred_x = group['pred_x']
        pred_y = group['pred_y']
        residual_x = group['residual_x']
        residual_y = group['residual_y']
        cpe_pred_x = group['cpe_pred_x']
        cpe_pred_y = group['cpe_pred_y']
        cpe_resi_x = group['cpe_resi_x']
        cpe_resi_y = group['cpe_resi_y']

        
        x = die_x * step_pitch_x + map_shift_x
        y = die_y * step_pitch_y + map_shift_y
        rx = coordinate_x
        ry = coordinate_y

        X_dx = np.vstack([
            np.ones(len(x)), x/1e6, -y/1e6, (x**2)/1e12, (x*y)/1e12, (y**2)/1e12, (x**3)/1e15, (x**2*y)/1e15, (x*y**2)/1e15, (y**3)/1e15, 
            rx/1e6, -ry/1e6, (rx**2)/1e9, (rx*ry)/1e9, (ry**2)/1e9, (rx**3)/1e12, (rx**2*ry)/1e12, (rx*ry**2)/1e12, (ry**3)/1e12
        ]).T
        X_dy = np.vstack([
            np.ones(len(y)), y/1e6, x/1e6, (y**2)/1e12, (y*x)/1e12, (x**2)/1e12, (y**3)/1e15, (y**2*x)/1e15, (y*x**2)/1e15, (x**3)/1e15,
            ry/1e6, rx/1e6, (ry**2)/1e9, (ry*rx)/1e9, (rx**2)/1e9, (ry**3)/1e12, (ry**2*rx)/1e12, (ry*rx**2)/1e12
        ]).T

        # 현재 UNIQUE_ID에 해당하는 MRC 데이터만 선택
        mrc_k_odd = df_mrc_input[
            (df_mrc_input['UNIQUE_ID'] == unique_id) & 
            (df_mrc_input['K PARA'].isin(['W1', 'W3', 'W5', 'W7', 'W9', 'W11', 'W13', 'W15', 'W17', 'W19', 
                                          'R3', 'R5', 'R7', 'R9', 'R11', 'R13', 'R15', 'R17', 'R19']))
        ]

        mrc_k_even = df_mrc_input[
            (df_mrc_input['UNIQUE_ID'] == unique_id) & 
            (df_mrc_input['K PARA'].isin(['W2', 'W4', 'W6', 'W8', 'W10', 'W12', 'W14', 'W16', 'W18', 'W20', 
                                          'R4', 'R6', 'R8', 'R10', 'R12', 'R14', 'R16', 'R18']))
        ]

        # 'K PARA'가 정확히 19개씩 있는지 확인
        if len(mrc_k_odd) != 19 or len(mrc_k_even) != 18:
            print(f"UNIQUE_ID {unique_id}의 MRC 데이터가 부족합니다.")
            continue

        # 'GPM' 값을 numpy 배열로 변환
        mrc_k_odd_values = mrc_k_odd['GPM'].values.astype(float)
        mrc_k_even_values = mrc_k_even['GPM'].values.astype(float)



        # mrc fitting (부호반대처리)
        mrc_fit_x = X_dx.dot(mrc_k_odd_values) * -1
        mrc_fit_y = X_dy.dot(mrc_k_even_values) * -1

        # X_reg 에서 mrc 빼주기
        X_reg_demrc = group['X_reg'] - mrc_fit_x
        Y_reg_demrc = group['Y_reg'] - mrc_fit_y

        # 결과 저장
        mrc_list.append(pd.DataFrame({

            'UNIQUE_ID' : unique_id,
            'STEPSEQ' : stepseq, 
            'LOT_ID' : lot_id,
            'Wafer' : wafer, 
            'P_EQPID' : p_eqpid,
            'Photo_PPID' : photo_ppid,
            'P_TIME' : p_time,
            'M_TIME' : m_time,
            'ChuckID' : chuckid,
            'ReticleID' : reticleid,
            'Base_EQP1' : base_eqp1,
            'TEST' : test,
            'DieX' : die_x,
            'DieY' : die_y,
            'MRC_X' : mrc_x,
            'MRC_Y' : mrc_y,
            'STEP_PITCH_X' : step_pitch_x,
            'STEP_PITCH_Y' : step_pitch_y,
            'MAP_SHIFT_X' : map_shift_x,
            'MAP_SHIFT_Y' : map_shift_y,
            'coordinate_X' : coordinate_x,
            'coordinate_Y' : coordinate_y,
            'wf_x' : wf_x,
            'wf_y' : wf_y,
            'X_reg' : x_reg,
            'Y_reg' : y_reg,
            'pred_x': pred_x,
            'pred_y': pred_y,
            'residual_x': residual_x,
            'residual_y': residual_y,
            'cpe_pred_x': cpe_pred_x,
            'cpe_pred_y': cpe_pred_y,
            'cpe_resi_x': cpe_resi_x,
            'cpe_resi_y': cpe_resi_y,            


            'mrc_fit_x': mrc_fit_x,
            'mrc_fit_y': mrc_fit_y,
            'X_reg_demrc': X_reg_demrc,
            'Y_reg_demrc': Y_reg_demrc,
        }))

    # mrc 결과 병합
    df_mrc_de = pd.concat(mrc_list, ignore_index=True)
    #df_mrc_de = pd.concat([df_rawdata.reset_index(drop=True), mrc_list], axis=1)

    return df_mrc_de



################################### MRC Decorrect #####################################################################

# 데이타 불러오기 
df_rawdata = pd.read_excel('output.xlsx', sheet_name='CPE_fit_res')
df_mrc_input = pd.read_excel('output.xlsx', sheet_name='MRC')

# mrc k값을 fitting하고 X_reg에서 빼주기 
df_mrc_de = kmrc_decorrect(df_rawdata, df_mrc_input)

# mrc_de 결과를 엑셀 파일에 저장
with pd.ExcelWriter('output.xlsx', engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df_mrc_de.to_excel(writer, sheet_name='MRC_decorrect', index=False)






print("모든 작업이 완료되었습니다.")


